In [1]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

import pandas as pd
import tensorflow as tf
from transformers import DistilBertTokenizerFast, TFDistilBertForSequenceClassification
import matplotlib.pyplot as plt
import numpy as np

tf.config.list_physical_devices('GPU')

2024-10-24 12:21:05.737009: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 12:21:05.756970: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-24 12:21:05.776221: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-24 12:21:05.781724: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-24 12:21:05.796627: I tensorflow/core/platform/cpu_feature_guar

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
test_df = pd.read_csv('pt1/test.csv')
test_labels = tf.convert_to_tensor(test_df['label'].values)

2024-10-24 12:21:10.453522: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:89:00.0, compute capability: 7.0


In [3]:
model = TFDistilBertForSequenceClassification.from_pretrained('sentiment_model')
tokenizer = DistilBertTokenizerFast.from_pretrained('sentiment_model')
test_encodings = tokenizer(
        test_df['text'].tolist(),  # Text column from CSV
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='tf'
    )

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at sentiment_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [4]:
test_predictions = model.predict(test_encodings.data)
predicted_labels = tf.argmax(test_predictions.logits, axis=1).numpy()

782/782 [==============================] - 42s 49ms/step


In [6]:
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels)
recall = recall_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels)

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test Recall: {recall:.4f}')
print(f'Test F1 Score: {f1:.4f}')

Test Accuracy: 0.8773
Test Precision: 0.8708
Test Recall: 0.8860
Test F1 Score: 0.8783


In [7]:
model = TFDistilBertForSequenceClassification.from_pretrained('sentiment_model-ablation')
tokenizer = DistilBertTokenizerFast.from_pretrained('sentiment_model-ablation')
test_encodings = tokenizer(
        test_df['text'].tolist(),  # Text column from CSV
        truncation=True,
        padding=True,
        max_length=128,
        return_tensors='tf'
    )

Some layers from the model checkpoint at sentiment_model-ablation were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at sentiment_model-ablation and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
test_predictions = model.predict(test_encodings.data)
predicted_labels = tf.argmax(test_predictions.logits, axis=1).numpy()

782/782 [==============================] - 40s 49ms/step


In [9]:
accuracy = accuracy_score(test_labels, predicted_labels)
precision = precision_score(test_labels, predicted_labels)
recall = recall_score(test_labels, predicted_labels)
f1 = f1_score(test_labels, predicted_labels)

print(f'Test Accuracy: {accuracy:.4f}')
print(f'Test Precision: {precision:.4f}')
print(f'Test Recall: {recall:.4f}')
print(f'Test F1 Score: {f1:.4f}')

Test Accuracy: 0.8675
Test Precision: 0.8502
Test Recall: 0.8922
Test F1 Score: 0.8707
